In [1]:
"Recommendation System fo Retail Stores"

'Recommendation System fo Retail Stores'

In [2]:
import pandas as pd
import numpy as np

In [5]:
df_ = pd.read_csv("./online_retail.csv")

In [6]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
# Data Pre-Processing
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

In [8]:
df = retail_data_prep(df)

In [18]:
# Let's prepare data type for UK customers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

df_ge = df[df['Country'] == "Germany"]

df_ge.groupby(['Invoice', 'Description']). \
    agg({"Quantity": "sum"}). \
    unstack(). \
    fillna(0). \
    applymap(lambda x: 1 if x > 0 else 0).iloc[0:5, 0:5]

Quantity                                                                                                     
Description  3 STRIPEY MICE FELTCRAFT  50'S CHRISTMAS GIFT BAG LARGE  CHERRY BLOSSOM  DECORATIVE FLASK  DOLLY GIRL BEAKER  FLAMINGO LIGHTS
Invoice                                                                                                                                   
489526                              0                              0                                 0                  0                0
490395                              0                              0                                 0                  0                0
490563                              0                              0                                 0                  0                0
490564                              1                              0                                 0                  0                0
490682                              0                              0                                 0                  0                0

In [19]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [20]:
ge_inv_pro_df = create_invoice_product_df(df_ge)
ge_inv_pro_df.head()

Description   3 STRIPEY MICE FELTCRAFT   50'S CHRISTMAS GIFT BAG LARGE   CHERRY BLOSSOM  DECORATIVE FLASK   DOLLY GIRL BEAKER   FLAMINGO LIGHTS   I LOVE LONDON MINI BACKPACK   RED SPOT GIFT BAG LARGE   RED/WHITE DOT MINI CASES   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  12 IVORY ROSE PEG PLACE SETTINGS  12 MESSAGE CARDS WITH ENVELOPES  12 MINI TOADSTOOL PEGS  12 PENCIL SMALL TUBE WOODLAND  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE RED SPOTTY  12 PENCILS SMALL TUBE SKULL  12 PENCILS TALL TUBE POSY  12 PENCILS TALL TUBE RED RETROSPOT  12 PENCILS TALL TUBE RED SPOTTY  12 PENCILS TALL TUBE SKULLS  12 PENCILS TALL TUBE WOODLAND  12 PINK HEN+CHICKS IN BASKET  12 PINK ROSE PEG PLACE SETTINGS  12 RED ROSE PEG PLACE SETTINGS  16 PIECE CUTLERY SET PANTRY DESIGN  2 PICTURE BOOK EGGS EASTER BUNNY  2 PICTURE BOOK EGGS EASTER CHICKS  20 DOLLY PEGS RETROSPOT  200 BENDY SKULL STRAWS  200 RED + WHITE BENDY STRAWS  3 DRAWER ANTIQUE WHITE WOOD CABINET  3 HOOK HANGER MAGIC GARDEN  3 HOOK PHOTO SHELF ANTIQUE WHITE  3 PIECE SPACEBOY COOKIE CUTTER SET  3 RAFFIA RIBBONS 50'S CHRISTMAS   3 STRIPEY MICE FELTCRAFT  3 TIER CAKE TIN GREEN AND CREAM  3 TIER CAKE TIN RED AND CREAM  3 TIER SWEETHEART GARDEN SHELF  3 TRADITIONAl BISCUIT CUTTERS  SET  36 DOILIES DOLLY GIRL  36 DOILIES SPACEBOY DESIGN   36 DOILIES VINTAGE CHRISTMAS  36 FOIL HEART CAKE CASES  36 FOIL STAR CAKE CASES   36 PENCILS TUBE RED RETROSPOT  36 PENCILS TUBE RED SPOTTY  36 PENCILS TUBE SKULLS  36 PENCILS TUBE WOODLAND  3D CHRISTMAS STAMPS STICKERS   3D HEARTS  HONEYCOMB PAPER GARLAND  3D STICKERS CHRISTMAS STAMPS  3D STICKERS TRADITIONAL CHRISTMAS   3D TRADITIONAL CHRISTMAS STICKERS  3D VINTAGE CHRISTMAS STICKERS   4 GOLD FLOCK CHRISTMAS BALLS  4 LAVENDER BOTANICAL DINNER CANDLES  4 TRADITIONAL SPINNING TOPS  4 VANILLA BOTANICAL CANDLES  4 WILDFLOWER BOTANICAL CANDLES  5 HOOK HANGER MAGIC TOADSTOOL  5 HOOK HANGER RED MAGIC TOADSTOOL  50'S CHRISTMAS PAPER GIFT BAG  6 CHOCOLATE LOVE HEART T-LIGHTS  6 CROCHET STRAWBERRIES  6 EGG HOUSE PAINTED WOOD  6 GIFT TAGS 50'S CHRISTMAS   6 GIFT TAGS VINTAGE CHRISTMAS   6 RIBBONS ELEGANT CHRISTMAS   6 RIBBONS EMPIRE    6 RIBBONS RUSTIC CHARM  6 RIBBONS SHIMMERING PINKS   6 ROCKET BALLOONS   6 SEGMENT COLOURED SNACK BOWL   60 CAKE CASES DOLLY GIRL DESIGN  60 CAKE CASES VINTAGE CHRISTMAS  60 TEATIME FAIRY CAKE CASES  72 CAKE CASES DOLLY GIRL DESIGN  72 CAKE CASES VINTAGE CHRISTMAS  72 SWEETHEART FAIRY CAKE CASES  75 GREEN FAIRY CAKE CASES  ABC TREASURE BOOK BOX   ABSTRACT CIRCLE JOURNAL   ABSTRACT CIRCLES POCKET BOOK  ABSTRACT CIRCLES SKETCHBOOK   ACRYLIC GEOMETRIC LAMP  ACRYLIC HANGING JEWEL,PINK  ADULT APRON APPLE DELIGHT  ADVENT CALENDAR GINGHAM SACK  AFGHAN SLIPPER SOCK PAIR  AIRLINE BAG VINTAGE JET SET BROWN  AIRLINE BAG VINTAGE JET SET RED  AIRLINE BAG VINTAGE JET SET WHITE  AIRLINE BAG VINTAGE TOKYO 78  AIRLINE BAG VINTAGE WORLD CHAMPION   ALARM CLOCK BAKELIKE CHOCOLATE  ALARM CLOCK BAKELIKE GREEN  ALARM CLOCK BAKELIKE IVORY  ALARM CLOCK BAKELIKE ORANGE  ALARM CLOCK BAKELIKE PINK  ALARM CLOCK BAKELIKE RED   ALPHABET HEARTS STICKER SHEET  ALPHABET STENCIL CRAFT  ANGEL DECORATION PAINTED ZINC   ANIMAL STICKERS  ANT COPPER LIME BOUDICCA BRACELET  ANT COPPER PINK BOUDICCA BRACELET  ANT COPPER TURQ BOUDICCA BRACELET  ANTIQUE CREAM CUTLERY CUPBOARD  ANTIQUE CREAM CUTLERY SHELF   ANTIQUE GLASS DRESSING TABLE POT  ANTIQUE GLASS PEDESTAL BOWL  ANTIQUE SILVER T-LIGHT GLASS  ANTIQUE SILVER TEA GLASS ENGRAVED  ANTIQUE SILVER TEA GLASS ETCHED  APOTHECARY MEASURING JAR  APPLE BATH SPONGE  APRON APPLE DELIGHT  AREA PATROLLED METAL SIGN  ASS FLORAL PRINT MULTI SCREWDRIVER  ASSORTED BOTTLE TOP  MAGNETS   ASSORTED CAKES FRIDGE MAGNETS  ASSORTED CHEESE FRIDGE MAGNETS  ASSORTED COLOUR BIRD ORNAMENT  ASSORTED COLOUR LIZARD SUCTION HOOK  ASSORTED COLOUR MINI CASES  ASSORTED COLOUR SET 6 TEASPOONS  ASSORTED COLOUR SILK COIN PURSE  ASSORTED COLOUR T-LIGHT HOLDER  ASSORTED COLOURS SILK FAN 

In [21]:
ge_inv_pro_df = create_invoice_product_df(df_ge, id=True)
ge_inv_pro_df.head()

StockCode  10002  10125  10135  11001  15034  15036  15039  15044A  15044B  15044D  15056BL  15056N  15056P  15058A  15058B  15058C  15060B  16008  16011  16012  16014  16016  16033  16045  16052  16054  16156L  16156S  16161G  16161P  16161U  16168M  16169C  16169E  16169F  16169K  16169M  16169N  16169P  16218  16235  16236  16237  16238  16256B  16258A  16259  17003  17091A  17109D  20617  20619  20622  20652  20653  20654  20658  20661  20665  20668  20671  20672  20674  20675  20676  20677  20679  20681  20682  20684  20685  20689  20692  20696  20702  20703  20704  20705  20707  20711  20712  20713  20716  20717  20718  20719  20723  20724  20725  20726  20727  20728  20729  20730  20734  20738  20749  20750  20751  20752  20754  20755  20756  20758  20759  20760  20761  20762  20764  20766  20767  20768  20769  20770  20771  20772  20774  20794  20796  20798  20799  20816  20828  20829  20846  20857  20864  20866  20867  20868  20871  20886  20897  20902  20903  20914  20957  20961  20963  20964  20966  20967  20971  20972  20973  20974  20975  20977  20978  20979  20981  20982  20983  20984  20985  20990  20992  20997  20998  21000  21001  21002  21011  21014  21015  21025  21026  21027  21033  21034  21035  21039  21041  21042  21051  21054  21055  21056  21058  21059  21061  21062  21064  21067  21068  21069  21070  21077  21078  21080  21082  21084  21085  21086  21088  21090  21091  21092  21094  21096  21098  21099  21100  21106  21108  21110  21114  21115  21116  21117  21121  21122  21123  21124  21125  21126  21127  21128  21130  21135  21136  21137  21144  21153  21154  21155  21156  21157  21158  21159  21162  21163  21164  21165  21166  21169  21175  21179  21186  21188  21190  21191  21194  21196  21197  21202  21203  21204  21205  21206  21207  21208  21209  21210  21211  21212  21213  21215  21216  21217  21218  21219  21220  21221  21222  21223  21224  21226  21231  21232  21238  21239  21240  21241  21242  21243  21244  21245  21246  21247  21248  21249  21250  21251  21257  21258  21259  21260  21262  21263  21264  21269  21270  21272  21275  21276  21277  21278  21281  21282  21284  21285  21286  21289  21291  21292  21293  21307  21311  21314  21318  21319  21320  21324  21327  21328  21329  21330  21340  21348  21354  21355  21357  21358  21360  21361  21371  21374  21380  21381  21383  21389  21392  21393  21394  21395  21396  21397  21398  21399  21401  21402  21403  21408  21410  21411  21414  21415  21416  21424  21426  21427  21428  21429  21430  21431  21432  21439  21440  21441  21445  21446  21447  21452  21456  21458  21461  21462  21464  21467  21471  21472  21474  21479  21481  21482  21484  21485  21486  21488  21491  21494  21495  21497  21498  21499  21500  21503  21504  21506  21507  21508  21509  21519  21523  21524  21525  21527  21528  21529  21531  21533  21534  21535  21537  21539  21541  21542  21544  21555  21556  21558  21559  21561  21562  21563  21564  21576  21577  21578  21579  21580  21581  21586  21588  21590  21592  21609  21610  21611  21612  21614  21615  21618  21619  21620  21621  21622  21623  21624  21625  21628  21637  21640  21642  21645  21648  21650  21651  21652  21653  21655  21656  21658  21661  21662  21666  21667  21668  21669  21670  21671  21672  21673  21674  21675  21679  21680  21682  21683  21687  21690  21691  21693  21696  21697  21698  21700  21703  21704  21705  21706  21707  21708  21709  21710  21714  21715  21716  21717  21718  21719  21720  21721  21722  21723  21724  21725  21730  21731  21733  21735  21739  21741  21742  21746  21747  21749  21752  21754  21755  21756  21763  21768  21770  21773  21774  21775  21785  21786  21787  21788  21789  21790  21791  21801  21802  21803  21809  21811  21817  21826  21827  21828  21829  21830  21832  21833  21836  21837  21842  21843  21844  21845  21866  21870  21871  21872  21873  21874  21875  21877  21878  21880  21881  21882  21883  21884  21889  21890  21891  21892  21894  21896  21899  21903 

In [22]:
# Let's create function that ca help find stock code for further needs

df_ge['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])

def check_id(dataframe, stock_code):
    product_name = dataframe.loc[dataframe["StockCode"] == stock_code, "Description"].values[0]
    print(product_name)

check_id(df_ge, "21232")

STRAWBERRY CERAMIC TRINKET BOX


C:\Users\right\AppData\Local\Temp\ipykernel_16388\1837333974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ge['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])


In [23]:
from mlxtend.frequent_patterns import apriori, association_rules

# Possibilities of all product combinations
frequent_itemsets = apriori(ge_inv_pro_df, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False).head(10)

c:\Users\right\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
562,0.798479,(POST)
251,0.263625,(22326)
2602,0.242079,"(22326, POST)"
253,0.172370,(22328)
27,0.166033,(20719)
2657,0.163498,"(22328, POST)"
943,0.149556,"(POST, 20719)"
2505,0.138150,"(22326, 22328)"
317,0.135615,(22554)
142,0.133080,(21731)


In [24]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.sort_values("support", ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4079,(POST),(22326),0.798479,0.263625,0.242079,0.303175,1.150023,0.031580,1.056757,0.647338
4078,(22326),(POST),0.263625,0.798479,0.242079,0.918269,1.150023,0.031580,2.465668,0.177154
4189,(POST),(22328),0.798479,0.172370,0.163498,0.204762,1.187920,0.025864,1.040732,0.784993
4188,(22328),(POST),0.172370,0.798479,0.163498,0.948529,1.187920,0.025864,3.915263,0.191139
761,(20719),(POST),0.166033,0.798479,0.149556,0.900763,1.128099,0.016983,2.030711,0.136160


In [25]:
rules.sort_values("lift", ascending=False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
76564,"(84997D, 20677, 21240)","(21239, POST, 84997C)",0.011407,0.010139,0.010139,0.888889,87.666667,0.010024,8.908745,1.000000
76583,"(84997D, 20677)","(21239, POST, 84997C, 21240)",0.011407,0.010139,0.010139,0.888889,87.666667,0.010024,8.908745,1.000000
76582,"(84997D, 21239)","(20677, POST, 84997C, 21240)",0.011407,0.010139,0.010139,0.888889,87.666667,0.010024,8.908745,1.000000
57514,"(84997D, 20677)","(21239, POST, 84997C)",0.011407,0.010139,0.010139,0.888889,87.666667,0.010024,8.908745,1.000000
4737,(22563),(22562),0.011407,0.010139,0.010139,0.888889,87.666667,0.010024,8.908745,1.000000
77664,"(20676, 21245, 20675)","(20674, 21242, 21238, 21244)",0.011407,0.011407,0.011407,1.000000,87.666667,0.011277,inf,1.000000
77636,"(20676, 21245, 21238, 20675)","(20674, 21242, 21244)",0.011407,0.011407,0.011407,1.000000,87.666667,0.011277,inf,1.000000
77633,"(20674, 21242, 21238, 21244)","(20676, 21245, 20675)",0.011407,0.011407,0.011407,1.000000,87.666667,0.011277,inf,1.000000
76542,"(21239, POST, 84997C, 21240)","(84997D, 20677)",0.010139,0.011407,0.010139,1.000000,87.666667,0.010024,inf,0.998720
76543,"(20677, POST, 84997C, 21240)","(84997D, 21239)",0.010139,0.011407,0.010139,1.000000,87.666667,0.010024,inf,0.998720


In [26]:
check_id(df_ge, "21987")

PACK OF 6 SKULL PAPER CUPS


In [27]:
check_id(df_ge, "22747")

POPPY'S PLAYHOUSE BATHROOM


In [41]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []

    for i, product in sorted_rules["antecedents"].items():
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))

    recommendation_list = list({item for item_list in recommendation_list for item in item_list})

    return recommendation_list[:rec_count]

In [43]:
product_id = "21242"
bought_together = arl_recommender(rules, product_id, 3)

for items in bought_together:
    print(items)

POST
22367
21977
